# Lab | Data Aggregation and Filtering

Objective: Use Python programming techniques to filter and order data subsets, group data by one or more variables, and use aggregation functions to obtain insights, and derive meaningful conclusions from the data.

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing first data cleaning, formatting, and structuring.

Start by reading the dataset, standardizing column names, dealing with null values and creating the month column. You can do so by using the functions you developed in previous labs, or by reading the clean dataset you saved in the last lab `data structuring and combining`.

In [6]:
import pandas as pd
from thefunctions import lower_snake

# Read the dataset from the URL
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Standardize the column names using the lower_snake function
df = lower_snake(df)

# Drop rows with any null values
df = df.dropna()

# Filling null values with mean for numerical columns
numerical_cols = df.select_dtypes(include='number').columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Filling null values with mode for categorical columns
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Convert the 'effective_to_date' column to datetime format
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%m/%d/%y')

# Extract the month and create the 'month' column
df['month'] = df['effective_to_date'].dt.month



Then, answer the following questions:

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the median customer lifetime value by education level and gender.

5. Find the maximum, minimum, and average number of open complaints by state and policy type.

6. The marketing team wants to analyze the number of policies sold by state and month.

Present the data in a table where the months are arranged as columns and the states are arranged as rows.

7. Display a new DataFrame that contains the number of policies sold by month by state for the top 3 states with the highest number of policies sold.

*Hint:* 
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

8. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [7]:
#1
# Filter the DataFrame 
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]

# Display the new DataFrame
print(filtered_df)

       unnamed:_0 customer       state  customer_lifetime_value response  \
236           236  YJ16163      Oregon             11009.130490      Yes   
419           419  GW43195      Oregon             25807.063000      Yes   
442           442  IP94270     Arizona             13736.132500      Yes   
587           587  FJ28407  California              5619.689084      Yes   
1527         1527  TU53781      Oregon              8427.172486      Yes   
1809         1809  QO62792      Oregon              7840.165778      Yes   
2027         2027  TA66375      Oregon             11009.130490      Yes   
2125         2125  JC11405      Oregon             10963.957230      Yes   
2865         2865  FH77504  California             11009.130490      Yes   
3958         3958  KC26486     Arizona              8427.172486      Yes   
4475         4475  WP58340     Arizona             10963.957230      Yes   
4521         4521  LT72149      Nevada              5619.689084      Yes   
4566        

In [8]:
#2
# Filter the DataFrame to include only customers who responded "Yes" to the last marketing campaign
yes_response_df = df[df['response'] == 'Yes']

# Group the data by policy type and gender and calculate the average total_claim_amount for each group
average_claim_amount_by_policy_gender = yes_response_df.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()

# Display the result
print(average_claim_amount_by_policy_gender)

      policy_type gender  total_claim_amount
0  Corporate Auto      F          404.104008
1  Corporate Auto      M          391.370774
2   Personal Auto      F          457.056315
3   Personal Auto      M          472.489180
4    Special Auto      F          495.689401
5    Special Auto      M          368.523035


In [9]:
#3
# Group the data by state and calculate the total number of customers in each state
customer_count_by_state = df.groupby('state')['customer'].count().reset_index()

# Filter the results to include only states with more than 500 customers
states_with_more_than_500_customers = customer_count_by_state[customer_count_by_state['customer'] > 500]

# Display the result
print(states_with_more_than_500_customers)

        state  customer
0     Arizona       868
1  California      1552
3      Oregon      1281


In [10]:
#4
# Group the data by education and gender and calculate the median customer lifetime value for each group
median_clv_by_education_gender = df.groupby(['education', 'gender'])['customer_lifetime_value'].median().reset_index()

# Display the result
print(median_clv_by_education_gender)

              education gender  customer_lifetime_value
0              Bachelor      F              5752.485185
1              Bachelor      M              5797.377350
2               College      F              5642.596077
3               College      M              6005.847375
4                Doctor      F              5789.850900
5                Doctor      M              5843.989737
6  High School or Below      F              5978.754765
7  High School or Below      M              6081.481291
8                Master      F              5714.165863
9                Master      M              5512.342258


In [11]:
#5
# Group the data by state and policy type and calculate the maximum, minimum, and average number of open complaints for each group
complaints_stats_by_state_policy = df.groupby(['state', 'policy_type'])['number_of_open_complaints'].agg(['max', 'min', 'mean']).reset_index()

# Display the result
print(complaints_stats_by_state_policy)

         state     policy_type  max  min      mean
0      Arizona  Corporate Auto  5.0  0.0  0.456790
1      Arizona   Personal Auto  5.0  0.0  0.413374
2      Arizona    Special Auto  3.0  0.0  0.229167
3   California  Corporate Auto  4.0  0.0  0.296703
4   California   Personal Auto  5.0  0.0  0.398589
5   California    Special Auto  4.0  0.0  0.388889
6       Nevada  Corporate Auto  3.0  0.0  0.393617
7       Nevada   Personal Auto  5.0  0.0  0.422492
8       Nevada    Special Auto  3.0  0.0  0.285714
9       Oregon  Corporate Auto  4.0  0.0  0.331984
10      Oregon   Personal Auto  5.0  0.0  0.373444
11      Oregon    Special Auto  5.0  0.0  0.328571
12  Washington  Corporate Auto  4.0  0.0  0.467532
13  Washington   Personal Auto  5.0  0.0  0.409836
14  Washington    Special Auto  4.0  0.0  0.562500


In [12]:
#6
# Create a pivot table to analyze the number of policies sold by state and month
policies_by_state_month = df.pivot_table(index='state', columns='month', values='policy', aggfunc='count', fill_value=0)

# Display the result
print(policies_by_state_month)

month         1    2
state               
Arizona     453  415
California  826  726
Nevada      247  197
Oregon      676  605
Washington  207  191


In [13]:
#7
# Group the data by state and month and calculate the number of policies sold for each combination
policies_by_state_month = df.groupby(['state', 'month'])['policy'].count().reset_index()

# Find the top 3 states with the highest number of policies sold
top_3_states = policies_by_state_month.groupby('state')['policy'].sum().nlargest(3).index

# Filter the data to include only the top 3 states
top_3_states_data = policies_by_state_month[policies_by_state_month['state'].isin(top_3_states)]

# Pivot the data to create a new DataFrame with months as columns and states as rows
policies_by_month_top_3_states = top_3_states_data.pivot_table(index='state', columns='month', values='policy', fill_value=0)

# Display the result
print(policies_by_month_top_3_states)


month         1    2
state               
Arizona     453  415
California  826  726
Oregon      676  605


In [24]:
#8

# Filter the DataFrame to include only rows where the customer responded "Yes" to the last marketing campaign
response_column_name = 'number_of_open_complaints'  # Dataset representing the customer response
yes_response_df = df[df[response_column_name] == 'Yes']

# Get the list of columns representing the marketing channels
marketing_channel_columns = ['Sales Channel']

# Check which of the marketing channel columns are present in the DataFrame
existing_columns = [col for col in marketing_channel_columns if col in yes_response_df.columns]

# Use melt to unpivot the data and create a table that shows the customer response rate by marketing channel
melted_data = pd.melt(yes_response_df, id_vars=[response_column_name], value_vars=existing_columns, var_name='marketing_channel', value_name='response_rate')